In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
ts1 = np.loadtxt("ts1.txt")
ts2 = np.loadtxt("ts2.txt")
ts3 = np.loadtxt("ts3.txt")

In [3]:
import numpy as np
from Operations.CO_FirstCrossing import CO_FirstCrossing

def CO_glscf(y, alpha, beta, tau = 'tau'):
    """
    """
    # Set tau to first zero-crossing of the autocorrelation function with the input 'tau'
    if tau == 'tau':
        tau = CO_FirstCrossing(y, 'ac', 0, 'discrete')
    
    # Take magnitudes of time-delayed versions of the time series
    y1 = np.abs(y[:-tau])
    y2 = np.abs(y[tau:])


    p1 = np.mean(np.multiply((y1 ** alpha), (y2 ** beta)))
    p2 = np.multiply(np.mean(y1 ** alpha), np.mean(y2 ** beta))
    p3 = np.sqrt(np.mean(y1 ** (2*alpha)) - (np.mean(y1 ** alpha))**2)
    p4 = np.sqrt(np.mean(y2 ** (2*beta)) - (np.mean(y2 ** beta))**2)

    glscf = (p1 - p2) / (p3 * p4)

    return glscf    


In [11]:
CO_glscf(ts1, 0.9, 0.4, 3)

0.25284356429315435

In [13]:
import numpy as np
from Operations.CO_FirstCrossing import CO_FirstCrossing
from Operations.CO_glscf import CO_glscf

def CO_fzcglscf(y, alpha, beta, maxtau = None):
    """
    """
    N = len(y) # the length of the time series

    if maxtau is None:
        maxtau = N
    
    glscfs = np.zeros(maxtau)

    for i in range(1, maxtau+1):
        tau = i

        glscfs[i-1] = CO_glscf(y, alpha, beta, tau)
        if (i > 1) and (glscfs[i-1]*glscfs[i-2] < 0):
            # Draw a straight line between these two and look at where it hits zero
            out = i - 1 + glscfs[i-1]/(glscfs[i-1]-glscfs[i-2])
            return out
    
    return maxtau # if the function hasn't exited yet, set output to maxtau 


In [86]:
import warnings
import numpy as np

def DN_cv(x, k = 1):
    """
    Coefficient of variation

    Coefficient of variation of order k is sigma^k / mu^k (for sigma, standard
    deviation and mu, mean) of a data vector, x

    Parameters:
    ----------
    x (array-like): The input data vector
    k (int, optional): The order of coefficient of variation (k = 1 is default)

    Returns:
    -------
    float: The coefficient of variation of order k
    """
    if not isinstance(k, int) or k < 0:
        warnings.warn('k should probably be a positive integer')
        # Carry on with just this warning, though
    
    # Compute the coefficient of variation (of order k) of the data
    return (np.std(x, ddof=1) ** k) / (np.mean(x) ** k)


In [88]:
DN_cv(ts3)

26.269342687595913

In [113]:
DN_nlogL_norm(ts2)

1.2765413621752804

In [118]:
import numpy as np 

def DN_HighLowMu(y):
    """
    The highlowmu statistic.

    The highlowmu statistic is the ratio of the mean of the data that is above the
    (global) mean compared to the mean of the data that is below the global mean.

    Paramters:
    ----------
    y (array-like): The input data vector

    Returns:
    --------
    float: The highlowmu statistic.
    """
    mu = np.mean(y) # mean of data
    mhi = np.mean(y[y > mu]) # mean of data above the mean
    mlo = np.mean(y[y < mu]) # mean of data below the mean
    out = np.divide((mhi-mu), (mu-mlo)) # ratio of the differences

    return out


In [119]:
DN_HighLowMu(ts2)

1.0325203252032518

In [120]:
import numpy as np

def BF_SimpleBinner(xData, numBins):
    """
    Generate a histogram from equally spaced bins.
   
    Parameters:
    xData (array-like): A data vector.
    numBins (int): The number of bins.

    Returns:
    tuple: (N, binEdges)
        N (numpy.ndarray): The counts
        binEdges (numpy.ndarray): The extremities of the bins.
    """
    minX = np.min(xData)
    maxX = np.max(xData)
    
    # Linearly spaced bins:
    binEdges = np.linspace(minX, maxX, numBins + 1)
    N = np.zeros(numBins, dtype=int)
    
    for i in range(numBins):
        if i < numBins - 1:
            N[i] = np.sum((xData >= binEdges[i]) & (xData < binEdges[i+1]))
        else:
            # the final bin
            N[i] = np.sum((xData >= binEdges[i]) & (xData <= binEdges[i+1]))
    
    return N, binEdges


In [140]:
from scipy import stats

def DN_Cumulants(y, cumWhatMay = 'skew1'):
    
    """
    """

    if cumWhatMay == 'skew1':
        out = stats.skew(y)
    elif cumWhatMay == 'skew2':
        out = stats.skew(y, bias=False)
    elif cumWhatMay == 'kurt1':
        out = stats.kurtosis(y, fisher=False)
    elif cumWhatMay == 'kurt2':
        out = stats.kurtosis(y, bias=False, fisher=False)
    else:
        raise ValueError('Requested Unknown cumulant must be: skew1, skew2, kurt1, or kurt2')
    
    return out


In [141]:
DN_Cumulants(ts1, cumWhatMay='kurt1')

1.497362974297833

In [142]:
DN_Cumulants(ts1, cumWhatMay='kurt2')

1.4958467355321061

In [144]:
import numpy as np

def EN_ApEN(y, mnom = 1, rth = 0.2):
    """
    """
    r = rth * np.std(y, ddof=1) # threshold of similarity
    N = len(y) # time series length
    phi = np.zeros(2) # phi[0] = phi_m, phi[1] = phi_{m+1}

    for k in range(2):
        m = mnom+k # pattern length
        C = np.zeros(N - m + 1)
        # define the matrix x, containing subsequences of u
        x = np.zeros((N-m+1, m))

        # Form vector sequences x from the time series y
        x = np.array([y[i:i+m] for i in range(N - m + 1)])
        
        for i in range(N - m + 1):
            # Calculate the number of x[j] within r of x[i]
            d = np.abs(x - x[i])
            if m > 1:
                d = np.max(d, axis=1)
            C[i] = np.sum(d <= r) / (N - m + 1)

        phi[k] = np.mean(np.log(C))

    return phi[0] - phi[1]


In [149]:
EN_ApEN(ts3, 2, 0.4)

1.514020217676181

In [4]:
import numpy as np
from Operations.SB_CoarseGrain import SB_CoarseGrain

def SB_MotifThree(y, cgHow = 'quantile'):
    """
    Motifs in a coarse-graining of a time series to a 3-letter alphabet.

    Parameters:
    -----------
    y : np.ndarray
        Time series to analyze.
    cg_how : {'quantile', 'diffquant'}, optional
        The coarse-graining method to use:
        - 'quantile': equiprobable alphabet by time-series value
        - 'diffquant': equiprobably alphabet by time-series increments
        Default is 'quantile'.

    Returns:
    --------
    Dict[str, float]
        Statistics on words of length 1, 2, 3, and 4.
    """

    # Coarse-grain the data y -> yt
    numLetters = 3
    if cgHow == 'quantile':
        yt = SB_CoarseGrain(y, 'quantile', numLetters)
    elif cgHow == 'diffquant':
        yt = SB_CoarseGrain(np.diff(y), 'quantile', numLetters)
    else:
        raise ValueError(f"Unknown coarse-graining method {cgHow}")

    # So we have a vectory yt with entries in {1, 2, 3}
    N = len(yt) # length of the symbolized sequence derived from the time series

    # ------------------------------------------------------------------------------
    # Words of length 1
    # ------------------------------------------------------------------------------
    out1 = np.zeros(3)
    r1 = [np.where(yt == i + 1)[0] for i in range(3)]
    for i in range(3):
        out1[i] = len(r1[i]) / N

    out = {
        'a': out1[0], 'b': out1[1], 'c': out1[2],
        'h': f_entropy(out1)
    }

    # ------------------------------------------------------------------------------
    # Words of length 2
    # ------------------------------------------------------------------------------

    r1 = [r[:-1] if len(r) > 0 and r[-1] == N - 1 else r for r in r1]
    out2 = np.zeros((3, 3))
    r2 = [[r1[i][yt[r1[i] + 1] == j + 1] for j in range(3)] for i in range(3)]
    for i in range(3):
        for j in range(3):
            out2[i, j] = len(r2[i][j]) / (N - 1)

    out.update({
        'aa': out2[0, 0], 'ab': out2[0, 1], 'ac': out2[0, 2],
        'ba': out2[1, 0], 'bb': out2[1, 1], 'bc': out2[1, 2],
        'ca': out2[2, 0], 'cb': out2[2, 1], 'cc': out2[2, 2],
        'hh': f_entropy(out2)
    })

    # ------------------------------------------------------------------------------
    # Words of length 3
    # ------------------------------------------------------------------------------

    r2 = [[r[:-1] if len(r) > 0 and r[-1] == N - 2 else r for r in row] for row in r2]
    out3 = np.zeros((3, 3, 3))
    r3 = [[[r2[i][j][yt[r2[i][j] + 2] == k + 1] for k in range(3)] for j in range(3)] for i in range(3)]
    for i in range(3):
        for j in range(3):
            for k in range(3):
                out3[i, j, k] = len(r3[i][j][k]) / (N - 2)

    out.update({f'{chr(97+i)}{chr(97+j)}{chr(97+k)}': out3[i, j, k] 
                for i in range(3) for j in range(3) for k in range(3)})
    out['hhh'] = f_entropy(out3)

    # ------------------------------------------------------------------------------
    # Words of length 4
    # ------------------------------------------------------------------------------

    r3 = [[[r[:-1] if len(r) > 0 and r[-1] == N - 3 else r for r in plane] for plane in cube] for cube in r3]
    out4 = np.zeros((3, 3, 3, 3))
    r4 = [[[[r3[i][j][k][yt[r3[i][j][k] + 3] == l + 1] for l in range(3)] for k in range(3)] for j in range(3)] for i in range(3)]
    for i in range(3):
        for j in range(3):
            for k in range(3):
                for l in range(3):
                    out4[i, j, k, l] = len(r4[i][j][k][l]) / (N - 3)

    out.update({f'{chr(97+i)}{chr(97+j)}{chr(97+k)}{chr(97+l)}': out4[i, j, k, l] 
                for i in range(3) for j in range(3) for k in range(3) for l in range(3)})
    out['hhhh'] = f_entropy(out4)

    return out

# helper function 
def f_entropy(x):
    """Entropy of a set of counts, log(0) = 0"""
    return -np.sum(x[x > 0] * np.log(x[x > 0]))
